In [5]:
import requests
import re
from collections import Counter

ollama_url = 'http://localhost:11434/api/chat'

# CHAIN OF THOUGHT PROMPT ENGINEERING TECHNIQUE

In [6]:
def chain_of_thought_query(user_input: str, model: str='phi3:latest') -> str:
    '''chain of thought agent helping with mathmatical and reasoninng querries'''
    messages = [
        {'role':'system', 'content':'You are an intelligent AI assistent build for chain of thought tasks.'},
        {'role':'user', 'content': f'{question}, \n please explain step by step reasoning for the question provided.'}
    ]

    payload = {
        'model':model,
        'messages':messages,
        'options':{'temperature':0.0, 'num_predict': 300},
        'stream': False
    }

    response = requests.post(ollama_url, json=payload)
    response.raise_for_status()

    data = response.json()

    return data['message']['content']

if __name__=='__main__':
    question = 'If I have 5 apples and buy 3 more, how many do I have in total?'
    print(chain_of_thought_query(question))

Step 1: Identify the initial quantity - In this case, you start with 5 apples.

Step 2: Determine the additional amount being added - You are buying an extra 3 apples.

Step 3: Add the two quantities together to find the total number of apples - To do this, simply add the initial quantity (5) and the additional quantity (3).

Calculation: 5 + 3 = 8

So, after buying 3 more apples, you will have a total of 8 apples.


# TREE OF THOUGHT PROMPT ENGINEERING TECHNIQUE

In [7]:
def tree_of_thought(user_input:str, model:str='phi3:latest') -> str:
    '''tree of thought agent helping in branching technique'''

    messages=[
        {'role':'system', 'content':('''You are a careful and intelligent AI assistant. 
            For the following problem, think of multiple possible reasoning paths (branches), 
            explore each path step by step, and then choose the best final answer.'''
        )},
        {'role':'user', 'content':f'{question},\n please think in a tree of thought format and give the best final answer.'}
    ]

    payload = {
        'model':model,
        'messages':messages,
        'options':{'temperature':0.0, 'num_predict':300},
        'stream': False
    }

    response = requests.post(ollama_url, json=payload)
    response.raise_for_status()

    data = response.json()
    return data['message']['content']

if __name__ == '__main__':
    question = 'If I have 5 apples and buy 3 more, how many do I have in total?'
    print(tree_of_thought(question))

Branch 1: Buy Apples First (Path A)
Step 1: Start with initial number of apples = 5
Step 2: Add purchased apples to existing ones, so total bought apples = 3 + 0 (initially no additional purchases made yet). This gives us a subtotal of 5.
Step 3: Combine the starting amount and added purchase for final count, which is still equal to our initial number since we haven't completed any transactions just yet. So at this point in Path A, I have 5 apples (no change from start).

Branch 2: Buy Apples After Starting with Initial Amount (Path B)
Step 1: Start with the initial amount of apples = 5
Step 2: Add purchased apples to existing ones. So total bought apples remain at 3 + 0, giving us a subtotal of 8 when combined with starting number of apples. This is because we've already added our purchases in Path A and now are just confirming the final count after all transactions have been completed.
Step 3: The total amount remains unchanged from Step 2 since no additional actions were taken, so I

# PROMPT CHAINING PROMPT ENGINEERING TECHNIQUE

In [8]:
def call_ollama_for_every_prompt(messages, model:str='phi3:latest') -> str:
    '''calling ollama model for every prompt in chain'''
    payload = {
        'model':model,
        'messages': messages,
        'options':{'temperature': 0.0, 'num_pridict': 300},
        'stream': False
    }

    response = requests.post(ollama_url, json=payload)
    response.raise_for_status()

    data = response.json()
    return data['message']['content']

def prompt_chaining(question):

    '''understand the problem'''
    messages_step1 = [
        {'role':'system', 'content':'You are an AI that summarizes problems clearly.'},
        {'role':'user', 'content':f'explain this queation as what is asked \n {question}'}
    ]

    understanding = call_ollama_for_every_prompt(messages_step1)

    '''reason the problem'''
    messages_step2 = [
        {'role':'system', 'content':'You are an AI that reasons step by step.'},
        {'role':'user', 'content':f'Problem understanding:\n{understanding}\n\nSolve it step by step.'}
    ]

    reasoning = call_ollama_for_every_prompt(messages_step2)

    '''final answer only'''

    messages_step3 = [
        {'role':'system', 'content':'Give only the final answer, no explanation.'},
        {'role':'user', 'content':f'provide the final answer{reasoning}'}
    ]

    final_answer = call_ollama_for_every_prompt(messages_step3)

    return {
        'understanding': understanding,
        'reasoning': reasoning,
        'final_answer': final_answer
    }

if __name__ == '__main__':
    question = 'If I have 10 apples and give away 4, then buy 3 more, how many apples do I have?'
    result = prompt_chaining(question)

    print('understanding\n', result['understanding'])
    print('reasoning\n', result['reasoning'])
    print('final_answer\n', result['final_answer'])

understanding
 The question asks for the total number of apples remaining after giving some away and purchasing additional ones. Specifically: Starting with a certain quantity (10), subtracting those given to someone else (4), and finally adding back in new acquisitions (3). The task is to calculate this final count.
reasoning
 Step 1: Identify the initial number of apples, which is provided as 10. This represents our starting point before any transactions occur.

Step 2: Determine how many apples are given away to someone else; in this case, it's mentioned that we give away 4 apples from our original count. To find out the remaining number of apples after giving some away, subtract these 4 apples from the initial quantity (10 - 4 = 6).

Step 3: Now consider any additional acquisitions or purchases made; here we are told that an extra 3 apples have been acquired. To find out our new total count of apples after this purchase, add these newly obtained apples to the remaining quantity fro

# SELF CONSISTENCY PROMPT ENGINEERING TECHNIQUE

In [9]:
def extract_final_answer(text):
    patterns = [
        r"FINAL\s*ANSWER\s*[:\-]\s*<?\s*(\d+)\s*>?",
        r"Final\s*Answer\s*[:\-]\s*(\d+)",
        r"Answer\s*[:\-]\s*(\d+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return match.group(1)
    return None


def self_consistency(question, runs=5, model="phi3:latest"):
    answers = []

    for _ in range(runs):
        messages = [
            {"role": "system", "content": "You are an AI that reasons step by step."},
            {"role": "user", "content": (
                f"{question}\n"
                "Explain your reasoning step by step.\n"
                "On the last line, write:\n"
                "FINAL ANSWER: <number>"
            )}
        ]

        payload = {
            "model": model,
            "messages": messages,
            "options": {"temperature": 0.7, "num_predict": 200},
            "stream": False
        }

        r = requests.post(ollama_url, json=payload)
        r.raise_for_status()
        output = r.json()["message"]["content"]

        answer = extract_final_answer(output)
        if answer:
            answers.append(answer)

    if not answers:
        raise ValueError("No valid answers extracted from model outputs.")

    most_common_answer = Counter(answers).most_common(1)[0][0]

    return {
        "all_answers": answers,
        "final_answer": most_common_answer
    }


if __name__ == '__main__':
    question = 'If I have 5 apples and buy 3 more, how many apples do I have?'
    result = self_consistency(question)

    print('all answers:')
    for a in result['all_answers']:
        print('-',a)

    print('\nfinal (most consistent) answer:')
    print(result['final_answer'])

all answers:
- 8
- 8

final (most consistent) answer:
8


# REACT PROMPT ENGINEERING TECHNIQUE

In [10]:
def react_prompt(question, model="phi3:latest"):
    messages = [
        {
            "role": "system",
            "content": (
                "You are an AI that uses the ReAct pattern.\n"
                "Use this format strictly:\n\n"
                "Thought:\n"
                "Action:\n"
                "Observation:\n"
                "Thought:\n"
                "Final Answer:"
            )
        },
        {
            "role": "user",
            "content": question
        }
    ]

    payload = {
        "model": model,
        "messages": messages,
        "options": {"temperature": 0.3, "num_predict": 300},
        "stream": False
    }

    r = requests.post(ollama_url, json=payload)
    r.raise_for_status()
    return r.json()["message"]["content"]

if __name__ == "__main__":
    question = "If I have 10 apples and give away 4, then buy 3 more, how many apples do I have?"
    print(react_prompt(question))

Thought: To solve this problem, we need to perform simple arithmetic operations. First, subtract the number of given-away apples from the initial count, and then add the newly purchased ones. The ReAct pattern involves these steps without any deviation or additional information required.
Action: Calculate (10 - 4 + 3) using a calculator if necessary to ensure accuracy in arithmetic operations.
Observation: After performing the calculation, we find that I have 9 apples remaining.
Thought: With this observation confirmed by calculations, it's clear how many apples are left after giving some away and buying more. The ReAct pattern has been followed without any deviations or additional information needed beyond what was provided in the question itself.
Final Answer: I have 9 apples now.


# PAL (Program-Aided Language) PROMPT ENGINEERING TECHNIQUE

In [11]:
def pal_prompt(question, model="phi3:latest"):
    '''PAL Prompting (Program-Aided Language)
    PAL = Let the model write code to solve the problem, instead of reasoning only in text.

👉 The model:

Understands the problem

Writes a small Python program

Uses the program to get the correct answer

This is very powerful for math, logic, counting, and rules.'''
    messages = [
        {
            "role": "system",
            "content": (
                "You are an AI that solves problems by writing Python code. "
                "Only write valid Python code. Do not explain anything."
                "Do not use markdown. Do not use ```."

            )
        },
        {
            "role": "user",
            "content": f"""
Solve the following problem by writing Python code.

Problem:
{question}

Rules:
- Only output Python code
- Store the final answer in a variable called `result`
"""
        }
    ]

    payload = {
        "model": model,
        "messages": messages,
        "options": {"temperature": 0.0, "num_predict": 200},
        "stream": False
    }

    r = requests.post(ollama_url, json=payload)
    r.raise_for_status()

    code = r.json()["message"]["content"]
    return code

import re

def clean_python_code(code: str) -> str:
    """
    Removes markdown code fences like ```python ... ```
    """
    code = re.sub(r"```python", "", code)
    code = re.sub(r"```", "", code)
    return code.strip()


def execute_pal_code(code):
    local_vars = {}
    clean_code = clean_python_code(code)
    exec(clean_code, {}, local_vars)
    return local_vars.get("result")



if __name__ == "__main__":
    question = "If I have 5 apples and buy 3 more, how many apples do I have?"

    code = pal_prompt(question)
    print("GENERATED CODE:\n", code)

    answer = execute_pal_code(code)
    print("FINAL ANSWER:", answer)


GENERATED CODE:
 ```python
apples = 5 + 3
result = apples
print(result)
```
8
FINAL ANSWER: 8
